<a href="https://colab.research.google.com/github/KritikaIyer17/Song_Genre_Classifier/blob/master/Logistic_Regression_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
import re

In [ ]:
df = pd.read_csv("/content/drive/My Drive/data/lyrics1.csv")
df['ranker_genre'] = np.where((df['ranker_genre'] == 'screamo')|(df['ranker_genre'] == 'punk rock')|(df['ranker_genre'] == 'heavy metal'),'alt rock',df['ranker_genre'])
df['ranker_genre'] = np.where((df['ranker_genre']== 'rythm and blues'),'pop',df['ranker_genre'])
group = ['song', 'year', 'album', 'genre', 'artist', 'ranker_genre']
lyrics_by_song = df.sort_values(group).groupby(group).lyric.apply(' '.join).apply(lambda x: x.lower()).reset_index(name='lyric')
lyrics_by_song["lyric"] = lyrics_by_song['lyric'].str.replace(r'[^\w\s]','')

In [ ]:
import nltk
nltk.download('stopwords')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;|\n]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
lyrics = lyrics_by_song["lyric"]

In [ ]:
def clean_text(text):
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text) 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    return text
lyrics = lyrics.apply(clean_text)
lyrics_by_song["lyric"]=lyrics

In [ ]:
X = lyrics_by_song.lyric
y = lyrics_by_song.ranker_genre
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)
logreg = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', LogisticRegression(n_jobs=1, multi_class = "multinomial",  C=100000, solver = "sag", class_weight="balanced")),])
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print(accuracy_score(y_pred, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.8125167587279455
